In [4]:
import pandas as pd
import numpy as np
import datetime 
import torch
from datetime import date, datetime, timedelta
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, variance, when
import pyspark.sql.functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StandardScaler, MinMaxScaler,  VectorAssembler
from pyspark.sql.types import *
from pyspark.ml import Pipeline

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=c35f5e5211a5be1bf14dbe008aa0710b72429cc0a1ba0410aa0f40c2d84f0b95
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


### IMPORT DATA

In [5]:
# IMPORT DATA
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('car_data.csv', header=True, inferSchema=True)
df.show(3)

+-------+------+---+------------+---------+
|User ID|Gender|Age|AnnualSalary|Purchased|
+-------+------+---+------------+---------+
|    385|  Male| 35|       20000|        0|
|    681|  Male| 40|       43500|        0|
|    353|  Male| 49|       74000|        0|
+-------+------+---+------------+---------+
only showing top 3 rows



In [5]:
from datetime import datetime, timedelta

model_run_date = '06-22-2023'
[str(datetime.strptime(model_run_date, "%m-%d-%Y") - timedelta(days=30*x))[:10] for x in [0, 1, 2, 3, 4, 5, 6]]


['2023-06-22',
 '2023-05-23',
 '2023-04-23',
 '2023-03-24',
 '2023-02-22',
 '2023-01-23',
 '2022-12-24']

In [8]:
model_run_date = '2023-06-22'
forecast_days = 90
[str(datetime.strptime(model_run_date,"%Y-%m-%d") + timedelta(days=forecast_days*x))[:10] for x in [0, 1]]

['2023-06-22', '2023-09-20']

### SQL COMMANDS

In [6]:
#understand Parquet files + udf

In [7]:
# Register the DataFrame as a temporary table/view
df.createOrReplaceTempView("car_data")
above_30 = spark.sql('SELECT * FROM car_data cd where cd.Age >= 35') 
above_30.show()

+-------+------+---+------------+---------+
|User ID|Gender|Age|AnnualSalary|Purchased|
+-------+------+---+------------+---------+
|    385|  Male| 35|       20000|        0|
|    681|  Male| 40|       43500|        0|
|    353|  Male| 49|       74000|        0|
|    895|  Male| 40|      107500|        1|
|    846|Female| 47|       33500|        1|
|    219|Female| 46|      132500|        1|
|    588|  Male| 42|       64000|        0|
|    465|  Male| 41|       52000|        0|
|    686|  Male| 42|       80000|        0|
|    408|  Male| 47|       23000|        1|
|    118|Female| 42|      108000|        1|
|     90|  Male| 35|       75000|        0|
|    372|  Male| 35|       53000|        0|
|    926|  Male| 46|       79000|        1|
|     94|Female| 39|      134000|        1|
|    338|Female| 39|       51500|        0|
|    134|Female| 49|       39000|        1|
|    821|  Male| 54|       25500|        1|
|    294|Female| 41|       61500|        0|
|     20|  Male| 40|      107000

### EDA

In [154]:
df.printSchema()

root
 |-- User ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- AnnualSalary: integer (nullable = true)
 |-- Purchased: integer (nullable = true)



In [155]:
#null val count
for col_ in df.columns:
  print(f'{col_} N/A:', df.filter(col(col_).isNull()).count())

User ID N/A: 0
Gender N/A: 0
Age N/A: 0
AnnualSalary N/A: 0
Purchased N/A: 0


In [156]:
df.groupBy('Gender').count().show() #unique vals in gender

+------+-----+
|Gender|count|
+------+-----+
|Female|  516|
|  Male|  484|
+------+-----+



In [157]:
#Find correlation with outcome var
def corr_(var):
  for col_ in df.columns:
    if col_ != var: continue
    for col2 in df.columns:
      if col_==col2 or df.select(F.col(col_)).dtypes[0][1] == 'string' or df.select(F.col(col2)).dtypes[0][1] == 'string': continue
      else:print('Correlation ({}, {}):'.format(col_, col2), df.select(F.corr(col_, col2)).first()[0])

corr_('Purchased')

Correlation (Purchased, User ID): 0.015910931566033848
Correlation (Purchased, Age): 0.6160363973749042
Correlation (Purchased, AnnualSalary): 0.3649744312828344


In [158]:
df.describe().toPandas()

,summary,User ID,Gender,Age,AnnualSalary,Purchased
0,count,1000,1000,1000,1000,1000
1,mean,500.5,None,40.106,72689.0,0.402
2,stddev,288.8194360957494,None,10.707072681429098,34488.34186685011,0.490547282773676
3,min,1,Female,18,15000,0
4,max,1000,Male,63,152500,1


#Logit Modeling

In [143]:
df.printSchema()

root
 |-- User ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- AnnualSalary: integer (nullable = true)
 |-- Purchased: integer (nullable = true)



In [173]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('car_data.csv', header=True, inferSchema=True)
df.show(3)

# Categorize Columns
indexer = StringIndexer(inputCol='Gender', outputCol='Gender_encoded').fit(df)
df = indexer.transform(df).select([x if x != 'Gender' else 'Gender_encoded' for x in df.columns])

# Random .8/.2 split
train, test = df.randomSplit([.8, .2], seed=0)

# Vectorize
v_assembler = VectorAssembler(inputCols=[col for col in df.columns if col != 'Purchased'],
                              outputCol='features')
df = v_assembler.transform(df).select('features', 'Purchased')

# Scale
scaler = MinMaxScaler(inputCol='features', outputCol='features_scaled').fit(df)
df = scaler.transform(df).select('features_scaled', 'Purchased')

df.show()

# Fit and Train Model
lr = LogisticRegression(featuresCol='features_scaled', labelCol= 'Purchased')
lr.setParams(regParam=0.01, elasticNetParam=0.5)

pipeline = Pipeline(stages=[indexer, v_assembler, scaler, lr])
model = pipeline.fit(train)


+-------+------+---+------------+---------+
|User ID|Gender|Age|AnnualSalary|Purchased|
+-------+------+---+------------+---------+
|    385|  Male| 35|       20000|        0|
|    681|  Male| 40|       43500|        0|
|    353|  Male| 49|       74000|        0|
+-------+------+---+------------+---------+
only showing top 3 rows

+--------------------+---------+
|     features_scaled|Purchased|
+--------------------+---------+
|[0.38438438438438...|        0|
|[0.68068068068068...|        0|
|[0.35235235235235...|        0|
|[0.89489489489489...|        1|
|[0.66066066066066...|        0|
|[0.84584584584584...|        1|
|[0.21821821821821...|        1|
|[0.58758758758758...|        0|
|[0.08408408408408...|        0|
|[0.46446446446446...|        0|
|[0.68568568568568...|        0|
|[0.40740740740740...|        1|
|[0.78978978978978...|        0|
|[0.11511511511511...|        0|
|[0.11711711711711...|        1|
|[0.05305305305305...|        1|
|[0.08908908908908...|        0|
|[0.371

In [166]:
#Predict with model --> RAWPREDICTION IS VALUE BEFORE SIGMOID FUNCTION/SOFTMAX
pred = model.transform(test)
pred.show()

+-------+--------------+---+------------+---------+--------------------+--------------------+--------------------+--------------------+----------+
|User ID|Gender_encoded|Age|AnnualSalary|Purchased|            features|     features_scaled|       rawPrediction|         probability|prediction|
+-------+--------------+---+------------+---------+--------------------+--------------------+--------------------+--------------------+----------+
|     21|           1.0| 36|       62500|        0|[21.0,1.0,36.0,62...|[0.02002002002002...|[1.66424712854177...|[0.84080730600946...|       0.0|
|     22|           0.0| 56|      131500|        1|[22.0,0.0,56.0,13...|[0.02102102102102...|[-3.4950924495082...|[0.02945218841335...|       1.0|
|     29|           1.0| 36|       40500|        0|[29.0,1.0,36.0,40...|[0.02802802802802...|[2.23988930854211...|[0.90377483233897...|       0.0|
|     30|           1.0| 27|       58000|        0|[30.0,1.0,27.0,58...|[0.02902902902902...|[3.37918598563868...|[0.9

In [196]:
#ADD MORE EVALUATORS
evaluator = BinaryClassificationEvaluator(labelCol='Purchased')
areaUnderROC = evaluator.evaluate(pred, {evaluator.metricName: 'areaUnderROC'})
#areaUnderPR = 'areaUnderPR'
print(areaUnderROC)

0.9261607142857144


In [56]:
#Gridsearch to optimize Logistic Regression HyperParameters

In [ ]:
#Evaluate Model Performance